# Charger et sauvegarder des données

## Fichiers texte

In [7]:
# Lire un fichier
txt = sc.textFile("/etc/passwd")

In [8]:
# Cleanup
!rm -rf /tmp/my-passwd

# Écrire un fichier
txt.saveAsTextFile("/tmp/my-passwd")

# Ça semble créer plusieurs segments/parties
!ls -lah /tmp/my-passwd

total 20K
drwxrwxr-x.  2 bl   bl    160 Jun 22 09:33 .
drwxrwxrwt. 53 root root 1.3K Jun 22 09:33 ..
-rw-r--r--.  1 bl   bl   1.4K Jun 22 09:33 part-00000
-rw-rw-r--.  1 bl   bl     20 Jun 22 09:33 .part-00000.crc
-rw-r--r--.  1 bl   bl   1.4K Jun 22 09:33 part-00001
-rw-rw-r--.  1 bl   bl     20 Jun 22 09:33 .part-00001.crc
-rw-r--r--.  1 bl   bl      0 Jun 22 09:33 _SUCCESS
-rw-rw-r--.  1 bl   bl      8 Jun 22 09:33 ._SUCCESS.crc


In [9]:
# cleanup
!rm -rf /tmp/my-range

# générer un plus grand nombre d'éléments et sauvegarder
range = sc.parallelize(xrange(1000,9999))
range.saveAsTextFile("/tmp/my-range")

# regarder le nombre de fichier et leur taille
!ls -lah /tmp/my-range

total 68K
drwxrwxr-x.  2 bl   bl    240 Jun 22 09:33 .
drwxrwxrwt. 53 root root 1.3K Jun 22 09:33 ..
-rw-r--r--.  1 bl   bl    11K Jun 22 09:33 part-00000
-rw-rw-r--.  1 bl   bl     96 Jun 22 09:33 .part-00000.crc
-rw-r--r--.  1 bl   bl    11K Jun 22 09:33 part-00001
-rw-rw-r--.  1 bl   bl     96 Jun 22 09:33 .part-00001.crc
-rw-r--r--.  1 bl   bl    11K Jun 22 09:33 part-00002
-rw-rw-r--.  1 bl   bl     96 Jun 22 09:33 .part-00002.crc
-rw-r--r--.  1 bl   bl    11K Jun 22 09:33 part-00003
-rw-rw-r--.  1 bl   bl     96 Jun 22 09:33 .part-00003.crc
-rw-r--r--.  1 bl   bl      0 Jun 22 09:33 _SUCCESS
-rw-rw-r--.  1 bl   bl      8 Jun 22 09:33 ._SUCCESS.crc


## JSON

In [10]:
# Charger un fichier as-is
json_unparsed = sc.textFile("/mnt/ul/ul-mpo-pr-wls01/depot_journal/mpo-prod/mpo-web1-cluster-ms1/mpo-audit/audit.log")

json_unparsed.first()

u'{"typeEntree":"navigation","urlDestination":"https://monportail.ulaval.ca/","codeAcces":"SASAR12","date":"2016-06-22T00:00:17.275-04:00[America/New_York]","remoteAddr":"74.15.155.153","userAgentFamily":"DESKTOP","userAgentPlatform":"MAC_OS","userAgentPlatformVersion":"10_11_5","userAgentType":"CHROME","userAgentVersion":"51.0.2704.103","userAgent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36"}'

In [11]:
# Ensuite le convertir en vrai objet json
import json

json_parsed = json_unparsed.map(lambda x: json.loads(x))
json_parsed.first()


{u'codeAcces': u'SASAR12',
 u'date': u'2016-06-22T00:00:17.275-04:00[America/New_York]',
 u'remoteAddr': u'74.15.155.153',
 u'typeEntree': u'navigation',
 u'urlDestination': u'https://monportail.ulaval.ca/',
 u'userAgent': u'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36',
 u'userAgentFamily': u'DESKTOP',
 u'userAgentPlatform': u'MAC_OS',
 u'userAgentPlatformVersion': u'10_11_5',
 u'userAgentType': u'CHROME',
 u'userAgentVersion': u'51.0.2704.103'}

In [12]:
# Nombre d'éléments
json_parsed.count()

851

In [13]:
# Accéder à des éléments dans la structure json pour filtrer ou effectuer tout type d'opération

json_parsed.filter(lambda x: x['typeEntree'] == "http").count()

734

In [14]:
# Cleanup 
!rm -rf /tmp/my-json

# Sauvegarder notre json natif en fichier
json_parsed.map(lambda x: json.dumps(x)).saveAsTextFile("/tmp/my-json")

# Regarder le nombre de fichier et leur taille
!ls -lah /tmp/my-json

total 664K
drwxrwxr-x.  2 bl   bl    160 Jun 22 09:33 .
drwxrwxrwt. 53 root root 1.3K Jun 22 09:33 ..
-rw-r--r--.  1 bl   bl   327K Jun 22 09:33 part-00000
-rw-rw-r--.  1 bl   bl   2.6K Jun 22 09:33 .part-00000.crc
-rw-r--r--.  1 bl   bl   324K Jun 22 09:33 part-00001
-rw-rw-r--.  1 bl   bl   2.6K Jun 22 09:33 .part-00001.crc
-rw-r--r--.  1 bl   bl      0 Jun 22 09:33 _SUCCESS
-rw-rw-r--.  1 bl   bl      8 Jun 22 09:33 ._SUCCESS.crc


In [15]:
# Regarder l'allure d'une ligne
!head -n 1 /tmp/my-json/part-00000

{"userAgentType": "CHROME", "codeAcces": "SASAR12", "typeEntree": "navigation", "userAgentPlatform": "MAC_OS", "userAgentVersion": "51.0.2704.103", "urlDestination": "https://monportail.ulaval.ca/", "date": "2016-06-22T00:00:17.275-04:00[America/New_York]", "userAgent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36", "userAgentPlatformVersion": "10_11_5", "remoteAddr": "74.15.155.153", "userAgentFamily": "DESKTOP"}


## CSV

In [16]:
import csv
import StringIO

def loadRecord(line):
    """Parse a CSV line"""
    input = StringIO.StringIO(line)
    reader = csv.DictReader(input, fieldnames=["FirstName","LastName","Title","ReportsTo.Email","Birthdate","Description"])
    return reader.next()

employees = sc.textFile("sample.csv").map(loadRecord)
employees.collect()


[{'Birthdate': '1940-06-07Z',
  'Description': 'Self-described as "the top" branding guru on the West Coast',
  'FirstName': 'Tom',
  'LastName': 'Jones',
  'ReportsTo.Email': 'buyer@salesforcesample.com',
  'Title': 'Senior Director'},
 {'Birthdate': '',
  'Description': 'World-renowned expert in fuzzy logic design. ',
  'FirstName': 'Ian',
  'LastName': 'Dury',
  'ReportsTo.Email': 'cto@salesforcesample.com',
  'Title': 'Chief Imagineer'},
 {'Birthdate': None,
  'Description': None,
  'FirstName': 'Influential in technology purchases."',
  'LastName': None,
  'ReportsTo.Email': None,
  'Title': None}]

In [17]:
# Sauvegarder

def writeRecords(records):
    """Write out CSV lines"""
    output = StringIO.StringIO()
    writer = csv.DictWriter(output, fieldnames=["FirstName","LastName","Title","ReportsTo.Email","Birthdate","Description"])

    for record in records:
        writer.writerow(record)
        
    return [output.getvalue()]

# Cleanup 
!rm -rf /tmp/my-csv

# Sauvegarder 
employees.mapPartitions(writeRecords).saveAsTextFile("/tmp/my-csv")


# Regarder le nombre de fichier et leur taille
!ls -lah /tmp/my-csv


total 20K
drwxrwxr-x.  2 bl   bl    160 Jun 22 09:33 .
drwxrwxrwt. 53 root root 1.3K Jun 22 09:33 ..
-rw-r--r--.  1 bl   bl    229 Jun 22 09:33 part-00000
-rw-rw-r--.  1 bl   bl     12 Jun 22 09:33 .part-00000.crc
-rw-r--r--.  1 bl   bl     48 Jun 22 09:33 part-00001
-rw-rw-r--.  1 bl   bl     12 Jun 22 09:33 .part-00001.crc
-rw-r--r--.  1 bl   bl      0 Jun 22 09:33 _SUCCESS
-rw-rw-r--.  1 bl   bl      8 Jun 22 09:33 ._SUCCESS.crc


## SequenceFiles

In [18]:
data = sc.parallelize([("Panda", 3), ("Kay", 6), ("Snail", 2)])

# Cleanup 
!rm -rf /tmp/my-sequence

# Sauvegarder 
data.saveAsSequenceFile("/tmp/my-sequence")


# Regarder le nombre de fichier et leur taille
!ls -lah /tmp/my-sequence



total 36K
drwxrwxr-x.  2 bl   bl    240 Jun 22 09:33 .
drwxrwxrwt. 53 root root 1.3K Jun 22 09:33 ..
-rw-r--r--.  1 bl   bl     85 Jun 22 09:33 part-00000
-rw-rw-r--.  1 bl   bl     12 Jun 22 09:33 .part-00000.crc
-rw-r--r--.  1 bl   bl    103 Jun 22 09:33 part-00001
-rw-rw-r--.  1 bl   bl     12 Jun 22 09:33 .part-00001.crc
-rw-r--r--.  1 bl   bl    101 Jun 22 09:33 part-00002
-rw-rw-r--.  1 bl   bl     12 Jun 22 09:33 .part-00002.crc
-rw-r--r--.  1 bl   bl    103 Jun 22 09:33 part-00003
-rw-rw-r--.  1 bl   bl     12 Jun 22 09:33 .part-00003.crc
-rw-r--r--.  1 bl   bl      0 Jun 22 09:33 _SUCCESS
-rw-rw-r--.  1 bl   bl      8 Jun 22 09:33 ._SUCCESS.crc


## Chargement JSON avec Spark SQL

### Test 1

In [19]:
from pyspark.sql import HiveContext
hiveCtx = HiveContext(sc)

In [20]:
tweets = hiveCtx.read.json("tweets.json")
tweets.registerTempTable("tweets")

You must build Spark with Hive. Export 'SPARK_HIVE=true' and run build/sbt assembly


Py4JJavaError: An error occurred while calling None.org.apache.spark.sql.hive.HiveContext.
: java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient
	at org.apache.hadoop.hive.ql.session.SessionState.start(SessionState.java:522)
	at org.apache.spark.sql.hive.client.ClientWrapper.<init>(ClientWrapper.scala:204)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.spark.sql.hive.client.IsolatedClientLoader.createClient(IsolatedClientLoader.scala:249)
	at org.apache.spark.sql.hive.HiveContext.metadataHive$lzycompute(HiveContext.scala:327)
	at org.apache.spark.sql.hive.HiveContext.metadataHive(HiveContext.s

In [ ]:
results = hiveCtx.sql("SELECT user.name, text FROM tweets")
results.collect()

### Test 2

In [ ]:
audit = hiveCtx.read.json("/mnt/ul/ul-mpo-pr-wls01/depot_journal/mpo-prod/mpo-web1-cluster-ms1/mpo-audit/audit.log")
audit.registerTempTable("audit")

In [ ]:
results = hiveCtx.sql("SELECT UPPER(codeAcces) AS codeAcces, COUNT(*) AS nb FROM audit GROUP BY codeAcces ORDER BY nb DESC")
results.collect()